In [ ]:
!pip install 'ms-swift[all]' -U
!pip install swanlab
!pip install bitsandbytes

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.0 MB/s eta 0:00:00 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 MB 82.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.0/316.0 kB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 MB 67.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.6/325.6 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.4/462.4 kB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 155.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 74.2 MB/s eta 0:00:00
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*

In [2]:
#打开网址 https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_linux_amd64 下载后上传
!pwd
# 1. 强制创建目标目录（如果不存在的话）
!mkdir -p /root/.cache/huggingface/gradio/frpc

# 2. 将文件复制过去并改名为带版本号的名称
!cp /mnt/workspace/frpc_linux_amd64 /root/.cache/huggingface/gradio/frpc/frpc_linux_amd64_v0.3

# 3. 赋予可执行权限
!chmod +x /root/.cache/huggingface/gradio/frpc/frpc_linux_amd64_v0.3

# 4. 验证是否成功（如果显示 -rwxr-xr-x 且文件名正确就没问题了）
!ls -l /root/.cache/huggingface/gradio/frpc/frpc_linux_amd64_v0.3

/mnt/workspace
-rwxr-xr-x 1 root root 2097152 12月 21 12:02 /root/.cache/huggingface/gradio/frpc/frpc_linux_amd64_v0.3


In [ ]:
!swift web-ui --lang zh --share

run sh: `/usr/local/bin/python /usr/local/lib/python3.11/site-packages/swift/cli/web_ui.py --lang zh --share`
[INFO:swift] Successfully registered `/usr/local/lib/python3.11/site-packages/swift/llm/dataset/data/dataset_info.json`.
[INFO:swift] args: WebUIArguments(server_name='0.0.0.0', server_port=7860, share=True, lang='zh')
[INFO:swift] Start time of running main: 2025-12-19 14:24:43.590645
[INFO:swift] swift.__version__: 3.11.1
/usr/local/lib/python3.11/site-packages/swift/ui/llm_infer/llm_infer.py:145: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(elem_id='chatbot', elem_classes='control-height')
* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://f76a7a4f6392f762af.gradio.live


In [ ]:
# 先统计一下token的数量和分布
import json
from collections import Counter
from modelscope import AutoTokenizer

# 用 Qwen3 的 tokenizer 统计长度
tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen3-4B",
    trust_remote_code=True
)

def count_tokens(messages):
    text = ""
    for turn in messages:
        role = turn.get("role", "")
        content = turn.get("content", "")
        text += f"{role}: {content}\n"
    return len(tokenizer.encode(text))

input_path = "/mnt/workspace/sharegpt_output.jsonl"

lengths = []
idx = 0

print("Scanning dataset and collecting token lengths...")

with open(input_path, "r", encoding="utf-8") as fin:
    for line in fin:
        idx += 1
        data = json.loads(line)
        messages = data.get("messages", [])
        token_count = count_tokens(messages)
        lengths.append(token_count)

print(f"\nTotal samples: {len(lengths)}")
if not lengths:
    raise SystemExit("No samples found, check input file.")

# 基本统计
min_len = min(lengths)
max_len = max(lengths)
avg_len = sum(lengths) / len(lengths)

print("\n=== Basic stats ===")
print(f"Min tokens: {min_len}")
print(f"Max tokens: {max_len}")
print(f"Avg tokens: {avg_len:.2f}")

# 1）按区间统计分布（可根据 max_len 自动设定上限）
bin_size = 256  # 你可以改成 128/512 等
max_bin_edge = ((max_len // bin_size) + 1) * bin_size

hist = Counter((l // bin_size) * bin_size for l in lengths)

print("\n=== Histogram by bins (bin_size = {} tokens) ===".format(bin_size))
print("Bin_start\tBin_end\tCount\tRatio")
total = len(lengths)
for bin_start in range(0, max_bin_edge + 1, bin_size):
    bin_end = bin_start + bin_size - 1
    count = hist.get(bin_start, 0)
    ratio = count / total * 100
    if count > 0:
        print(f"{bin_start:4d}\t\t{bin_end:4d}\t{count:5d}\t{ratio:6.2f}%")

# 2）关注“尾部”区间（接近最长样本）
tail_threshold = int(max_len * 0.75)  # 比如后 25% 段
tail_lengths = [l for l in lengths if l >= tail_threshold]

print("\n=== Tail distribution (>= {} tokens) ===".format(tail_threshold))
print(f"Tail samples: {len(tail_lengths)} ({len(tail_lengths) / total * 100:.2f}% of all)")

tail_bin_size = 128
tail_hist = Counter((l // tail_bin_size) * tail_bin_size for l in tail_lengths)
print("Bin_start\tBin_end\tCount\tRatio (within tail)")
for bin_start in range(tail_threshold // tail_bin_size * tail_bin_size, max_bin_edge + 1, tail_bin_size):
    bin_end = bin_start + tail_bin_size - 1
    count = tail_hist.get(bin_start, 0)
    if len(tail_lengths) > 0:
        ratio = count / len(tail_lengths) * 100
    else:
        ratio = 0.0
    if count > 0:
        print(f"{bin_start:4d}\t\t{bin_end:4d}\t{count:5d}\t{ratio:6.2f}%")

# 3）候选 max_length 切点的“保留比例”分析
cut_points = [1024, 1536, 2048, 2560, 3072, 3584, 4096]

print("\n=== Cut-point analysis (how many samples kept if max_length = X) ===")
print("max_length\tKept\tRatio")
lengths_sorted = sorted(lengths)
for cp in cut_points:
    kept = sum(1 for l in lengths_sorted if l <= cp)
    ratio = kept / total * 100
    print(f"{cp:5d}\t\t{kept:5d}\t{ratio:6.2f}%")


In [ ]:
import json
from modelscope import AutoTokenizer

# 你可以用 Qwen2 或 Qwen3 的 tokenizer，统计长度都没问题
tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen3-4B",
    trust_remote_code=True
)

def count_tokens(messages):
    text = ""
    for turn in messages:
        role = turn.get("role", "")
        content = turn.get("content", "")
        text += f"{role}: {content}\n"
    return len(tokenizer.encode(text))

input_path = "/mnt/workspace/sharegpt_output.jsonl"
output_path = "/mnt/workspace/sharegpt_filtered.jsonl"
max_tokens = 2048

idx = 0
kept = 0
removed = 0

with open(input_path, "r", encoding="utf-8") as fin, open(output_path, "w", encoding="utf-8") as fout:
    for line in fin:
        idx += 1
        data = json.loads(line)
        messages = data.get("messages", [])
        token_count = count_tokens(messages)

        print(f"[{idx}] tokens={token_count}")

        if token_count <= max_tokens:
            fout.write(json.dumps(data, ensure_ascii=False) + "\n")
            kept += 1
        else:
            removed += 1

print("\n=== Summary ===")
print(f"Total: {idx}")
print(f"Kept: {kept}")
print(f"Removed: {removed}")


[1] tokens=2479
[2] tokens=2564
[3] tokens=2474
[4] tokens=2518
[5] tokens=1169
[6] tokens=1242
[7] tokens=1352
[8] tokens=1173
[9] tokens=1324
[10] tokens=1263
[11] tokens=1351
[12] tokens=1345
[13] tokens=1237
[14] tokens=1374
[15] tokens=1298
[16] tokens=1264
[17] tokens=1262
[18] tokens=3298
[19] tokens=1250
[20] tokens=3788
[21] tokens=3711
[22] tokens=3090
[23] tokens=2983
[24] tokens=3623
[25] tokens=3699
[26] tokens=3761
[27] tokens=3089
[28] tokens=3036
[29] tokens=3249
[30] tokens=2603
[31] tokens=2758
[32] tokens=3032
[33] tokens=3026
[34] tokens=3946
[35] tokens=3829
[36] tokens=3915
[37] tokens=4011
[38] tokens=2441
[39] tokens=2818
[40] tokens=2260
[41] tokens=1350
[42] tokens=2743
[43] tokens=1293
[44] tokens=1340
[45] tokens=1192
[46] tokens=1430
[47] tokens=1288
[48] tokens=1290
[49] tokens=1276
[50] tokens=2646
[51] tokens=1177
[52] tokens=1338
[53] tokens=2856
[54] tokens=1274
[55] tokens=1441
[56] tokens=1172
[57] tokens=944
[58] tokens=1102
[59] tokens=1241
[60] to

In [ ]:
# 切分训练集，验证集和测试集
import random
from pathlib import Path

src_path = Path("/mnt/workspace/sharegpt_filtered.jsonl")
lines = src_path.read_text(encoding="utf-8").splitlines()
n = len(lines)
print(f"Total samples: {n}")

# 按比例划分
train_ratio = 0.8   # 80% 训练
val_ratio = 0.1     # 10% 验证
test_ratio = 0.1    # 10% 测试

assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6

random.seed(42)
random.shuffle(lines)

train_end = int(n * train_ratio)
val_end = train_end + int(n * val_ratio)

train = lines[:train_end]
val = lines[train_end:val_end]
test = lines[val_end:]

print(f"Train: {len(train)}, Val: {len(val)}, Test: {len(test)}")

Path("/mnt/workspace/train.jsonl").write_text("\n".join(train), encoding="utf-8")
Path("/mnt/workspace/val.jsonl").write_text("\n".join(val), encoding="utf-8")
Path("/mnt/workspace/test.jsonl").write_text("\n".join(test), encoding="utf-8")


In [ ]:
!CUDA_VISIBLE_DEVICES=0 \
 swift sft \
    --model Qwen/Qwen3-4B \
    --train_type lora \
   #  --dataset /mnt/workspace/sharegpt_filtered.jsonl \
    --torch_dtype bfloat16 \
    --bnb_4bit_compute_dtype bfloat16 \
    --bnb_4bit_quant_type nf4 \
    --bnb_4bit_use_double_quant true \
    --quant_method bnb \
    --quant_bits 4 \
    --dataset /mnt/workspace/train.jsonl \
    --eval_dataset /mnt/workspace/val.jsonl \
    --num_train_epochs 1 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --learning_rate 1e-4 \
    --lora_rank 8 \
    --lora_alpha 32 \
    --target_modules all-linear \
    --gradient_accumulation_steps 16 \
    --eval_steps 50 \
    --save_steps 50 \
    --save_total_limit 2 \
    --logging_steps 5 \
    --max_length 2048 \
    --output_dir output \
    --warmup_ratio 0.05 \
    --dataloader_num_workers 4 \
    --model_author yiming \
    --model_name ai-person \
    --report_to swanlab \
    --swanlab_project swift-Qwen4b-dichan

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/usr/local/lib/python3.11/site-packages/torch/cuda/__init__.py:56: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
run sh: `/usr/local/bin/python /usr/local/lib/python3.11/site-packages/swift/cli/sft.py --model Qwen/Qwen3-8B --train_type lora --dataset /mnt/workspace/sharegpt_filtered.jsonl --torch_dtype bfloat16 --bnb_4bit_compute_dtype bfloat16 --bnb_4bit_quant_type nf4 --bnb_4bit_use_double_quant true --quant_method bnb --quant_bits 4 --num_train_epochs 1 --per_device_train_batch_size 1 --per_device_eval_batch_size 1 --learning_rate 1e-4 --lora_rank 8 --lora_alpha 32 --target_modules all-linear --gradient_accumulation_steps 16 --eval_steps 50 --save_steps 50 --save_total_limit 2 --logging_steps 5 --max_length 2048 --output_dir output --warmup_ratio 0.05 --dataloader_num_workers 4 --mo

In [ ]:
#升级版本，适合正式训练
!CUDA_VISIBLE_DEVICES=0 \
 swift sft \
    --model Qwen/Qwen3-4B \
    --train_type lora \
   #  --dataset /mnt/workspace/sharegpt_filtered.jsonl \
    --torch_dtype bfloat16 \
    --bnb_4bit_compute_dtype bfloat16 \
    --bnb_4bit_quant_type nf4 \
    --bnb_4bit_use_double_quant true \
    --quant_method bnb \
    --quant_bits 4 \
    --dataset /mnt/workspace/train.jsonl \
    --eval_dataset /mnt/workspace/val.jsonl \
    --num_train_epochs 3 \                       # 修改1：1 -> 3，提升收敛质量（贴近 Swift 默认推荐值）
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --learning_rate 1e-4 \
    --lora_rank 8 \
    --lora_alpha 32 \
    --target_modules all-linear \
    --gradient_accumulation_steps 32 \           # 修改2：16 -> 32，增大全局 batch，略改善稳定性，对显存影响极小
    --eval_steps 100 \                           # 修改3：50 -> 100，减轻评估开销，显存压力微减
    --save_steps 200 \                           # 修改4：50 -> 200，减少存盘频率（对显存无影响）
    --save_total_limit 2 \
    --logging_steps 10 \                         # 修改5：5 -> 10，减少日志频率（对显存无影响）
    --max_length 4096 \                          # 修改6：2048 -> 4096，满足你必须 4096 的需求（显存压力↑↑）
    --gradient_checkpointing true \              # 修改7：新增，激活重计算，显存占用大幅下降，计算量↑
    --output_dir output \
    --warmup_ratio 0.05 \
    --dataloader_num_workers 2 \                 # 修改8：4 -> 2，减轻 CPU IO 压力，对显存无影响
    --model_author yiming \
    --model_name ai-person \
    --report_to swanlab \
    --swanlab_project swift-Qwen4b-dichan

In [ ]:
# --adapters output/v0-20251221-122053/checkpoint-12 \

def chat(prompt):
    cmd = f"""
    CUDA_VISIBLE_DEVICES=0 swift infer \
        --model Qwen/Qwen3-8B \
        --prompt "{prompt}" \
        --temperature 0 \
        --max_new_tokens 2048
    """
    !{cmd}

chat("你是谁？")



In [15]:
!cd /mnt/workspace && zip -r output.zip output #压缩lora

  adding: output/ (stored 0%)
  adding: output/v0-20251221-122053/ (stored 0%)
  adding: output/v0-20251221-122053/runs/ (stored 0%)
  adding: output/v0-20251221-122053/runs/events.out.tfevents.1766290997.eais-bjh1ari71pyj7xswquht-0.248.0 (deflated 65%)
  adding: output/v0-20251221-122053/images/ (stored 0%)
  adding: output/v0-20251221-122053/images/train_epoch.png (deflated 10%)
  adding: output/v0-20251221-122053/images/train_token_acc.png (deflated 11%)
  adding: output/v0-20251221-122053/images/train_train_steps_per_second.png (deflated 26%)
  adding: output/v0-20251221-122053/images/train_train_loss.png (deflated 25%)
  adding: output/v0-20251221-122053/images/train_loss.png (deflated 11%)
  adding: output/v0-20251221-122053/images/train_train_samples_per_second.png (deflated 24%)
  adding: output/v0-20251221-122053/images/train_train_runtime.png (deflated 23%)
  adding: output/v0-20251221-122053/images/train_total_flos.png (deflated 25%)
  adding: output/v0-20251221-122053/image

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 8%)
  adding: output/v0-20251221-122053/checkpoint-12/adapter_model.safetensors (deflated 8%)
  adding: output/v0-20251221-122053/checkpoint-12/args.json (deflated 73%)
  adding: output/v0-20251221-122053/args.json (deflated 73%)
  adding: output/v0-20251221-122053/logging.jsonl (deflated 65%)


In [1]:
!cd /mnt/workspace && unzip -o output.zip #解压


Archive:  output.zip
   creating: output/
   creating: output/v0-20251221-122053/
   creating: output/v0-20251221-122053/runs/
  inflating: output/v0-20251221-122053/runs/events.out.tfevents.1766290997.eais-bjh1ari71pyj7xswquht-0.248.0  
   creating: output/v0-20251221-122053/images/
  inflating: output/v0-20251221-122053/images/train_epoch.png  
  inflating: output/v0-20251221-122053/images/train_token_acc.png  
  inflating: output/v0-20251221-122053/images/train_train_steps_per_second.png  
  inflating: output/v0-20251221-122053/images/train_train_loss.png  
  inflating: output/v0-20251221-122053/images/train_loss.png  
  inflating: output/v0-20251221-122053/images/train_train_samples_per_second.png  
  inflating: output/v0-20251221-122053/images/train_train_runtime.png  
  inflating: output/v0-20251221-122053/images/train_total_flos.png  
  inflating: output/v0-20251221-122053/images/train_learning_rate.png  
  inflating: output/v0-20251221-122053/images/train_grad_norm.png  
   cre

In [4]:
#不用cuda，用cpu merge lora层否则直接爆显存

!CUDA_VISIBLE_DEVICES="" swift merge_lora \
    --model Qwen/Qwen3-8B \
    --adapters /mnt/workspace/output/v0-20251221-122053/checkpoint-12 \
    --output_dir /mnt/workspace/merged_model



run sh: `/usr/local/bin/python /usr/local/lib/python3.11/site-packages/swift/cli/merge_lora.py --model Qwen/Qwen3-8B --adapters /mnt/workspace/output/v0-20251221-122053/checkpoint-12 --output_dir /mnt/workspace/merged_model`
[INFO:swift] Successfully registered `/usr/local/lib/python3.11/site-packages/swift/llm/dataset/data/dataset_info.json`.
[INFO:swift] Successfully loaded /mnt/workspace/output/v0-20251221-122053/checkpoint-12/args.json.
[INFO:swift] rank: -1, local_rank: -1, world_size: 1, local_world_size: 1
[INFO:swift] Downloading the model from ModelScope Hub, model_id: Qwen/Qwen3-8B
[INFO:modelscope] Got 10 files, start to download ...
Processing 10 items:   0%|                          | 0.00/10.0 [00:00<?, ?it/s]

































Processing 10 items:  10%|█▉                 | 1.00/10.0 [00:00<00:04, 2.16it/s]





























Processing 10 items:  40%|███████▌           | 4.00/10.0 [00:00<00:00, 8.57it/s]




























Processing 10 

In [5]:
#用CPU 直接把模型量化为q4
!CUDA_VISIBLE_DEVICES="" swift export \
    --model /mnt/workspace/merged_model \
    --quant_method bnb \
    --quant_bits 4 \
    --output_dir /mnt/workspace/qwen3-8b-lora-q4


run sh: `/usr/local/bin/python /usr/local/lib/python3.11/site-packages/swift/cli/export.py --model /mnt/workspace/merged_model --quant_method bnb --quant_bits 4 --output_dir /mnt/workspace/qwen3-8b-lora-q4`
[INFO:swift] Successfully registered `/usr/local/lib/python3.11/site-packages/swift/llm/dataset/data/dataset_info.json`.
[INFO:swift] Successfully loaded /mnt/workspace/merged_model/args.json.
[INFO:swift] rank: -1, local_rank: -1, world_size: 1, local_world_size: 1
[INFO:swift] Setting args.lazy_tokenize: False
[INFO:swift] args.output_dir: `/mnt/workspace/qwen3-8b-lora-q4`
[INFO:swift] Global seed set to 42
[INFO:swift] args: ExportArguments(use_ray=False, ray_exp_name=None, device_groups=None, model='/mnt/workspace/merged_model', model_type='qwen3', model_revision=None, task_type='causal_lm', torch_dtype=torch.bfloat16, attn_impl=None, new_special_tokens=[], num_labels=None, problem_type=None, rope_scaling=None, device_map=None, max_memory={}, max_model_len=None, local_repo_path=

In [6]:
with open("input.txt", "w") as f:
    f.write("你是谁？")


In [7]:
!swift infer \
    --model /mnt/workspace/qwen3-8b-lora-q4 \
    --local_model true \
    --input_file input.txt


run sh: `/usr/local/bin/python /usr/local/lib/python3.11/site-packages/swift/cli/infer.py --model /mnt/workspace/qwen3-8b-lora-q4 --local_model true --input_file input.txt`
[INFO:swift] Successfully registered `/usr/local/lib/python3.11/site-packages/swift/llm/dataset/data/dataset_info.json`.
[INFO:swift] Successfully loaded /mnt/workspace/qwen3-8b-lora-q4/args.json.
[INFO:swift] rank: -1, local_rank: -1, world_size: 1, local_world_size: 1
[INFO:swift] Setting args.eval_human: True
[INFO:swift] Setting args.lazy_tokenize: False
[INFO:swift] args.result_path: /mnt/workspace/qwen3-8b-lora-q4/infer_result/20251221-150127.jsonl
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/swift/cli/infer.py", line 5, in <module>
    infer_main()
  File "/usr/local/lib/python3.11/site-packages/swift/llm/infer/infer.py", line 309, in infer_main
    return SwiftInfer(args).main()
           ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/swift/llm/infer/i